In [1]:
import os
import sys 
import pandas as pd
from train_models import train_dl_model, train_transf_chik, LOOK_BACK, PREDICT_N

sys.path.append('../')
from lstm import apply_model

df = pd.read_csv('s_cities.csv', index_col = 'Unnamed: 0')

#df = df.loc[~df.geocode.isin([3135050, 3143302, 1716109, 3304102, 3141405, 2905008, 29005008])]

#df = df.loc[df.geocode != 3168606]
#df = df.loc[]
#df = df.iloc[:1]
df.head()

,geocode,muni_name,year,peak_week,beta,gamma,R0,total_cases,alpha,sum_res,ep_ini,ep_end,ep_dur,uf,state,n_last_ep,year_first_ep,year_last_ep_before_2023,start_train_chik,end_train_chik
30,3304102,Porciúncula,2023,7.408430,0.742929,0.301735,2.462195,510.383085,0.593858,0.746891,202302,202316,14,33,RJ,3,2016,2020,2015-10-01,2020-11-01
8,2303501,Cascavel,2023,21.031323,0.448789,0.300973,1.491123,196.764406,0.329365,1.124633,202302,202339,37,23,CE,3,2017,2022,2016-10-01,2022-11-01
86,2311405,Quixeramobim,2023,18.375030,0.534700,0.304761,1.754491,466.379506,0.430034,0.930290,202302,202332,30,23,CE,3,2016,2022,2015-10-01,2022-11-01
88,2306801,Jaguaribara,2023,16.405845,0.742924,0.308515,2.408069,302.646554,0.584730,1.125556,202309,202325,16,23,CE,1,2017,2017,2016-10-01,2017-11-01
122,2305407,Icó,2023,13.682780,0.429925,0.300100,1.432607,87.124092,0.301972,1.434495,202302,202334,32,23,CE,2,2017,2022,2016-10-01,2022-11-01


### Train models

In [2]:
%%time
import pandas as pd
from train_models import train_dl_model, train_transf_chik
import os

end_date = '2023-11-01'
RETRAIN = True

for city, ini_date, end_date_train in zip(
    df.geocode, df.start_train_chik, df.end_train_chik):
    print(f"Training {city} model")

    # train the dengue model
    if RETRAIN or not os.path.exists(f'../saved_models/lstm/trained_{city}_dengue_msle.keras'):
        train_dl_model(city,   doenca = 'dengue', end_date_train = '2022-11-01' , end_date = end_date, plot = False)
    
        
    # apply transfer
    if  RETRAIN or not os.path.exists(f'../saved_models/lstm/trained_{city}_chik_tranf_msle.keras'):
        train_transf_chik(city, ini_date = ini_date, end_date_train = end_date_train, end_date = end_date, plot =False)
    
    
    
    # train the chik model
    if RETRAIN or not os.path.exists(f'../saved_models/lstm/trained_{city}_chik_msle.keras'):
        train_dl_model(city,   doenca = 'chik', end_date_train = '2022-11-01' , end_date = end_date, plot = False)


Training 3304102 model
Compilation Time :  0.002809762954711914
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12, 87)]          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 12, 8)            6144      
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 12, 8)             0         
                                                                 
 lstm_1 (LSTM)               (None, 12, 8)             544       
                                                                 
 dropout_2 (Dropout)         (None, 12, 8)             0         
                                                                 
 lstm_2 (LSTM)               (None, 8)                 544     

2024-01-22 23:44:37.458609: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-01-22 23:44:37.460545: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1780] (One-time warning): Not using XLA:CPU for cluster.

If you want XLA:CPU, do one of the following:

 - set the TF_XLA_FLAGS to include "--tf_xla_cpu_global_jit", or
 - set cpu_global_jit to true on this session's OptimizerOptions, or
 - use experimental_jit_scope, or
 - use tf.function(jit_compile=True).

To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a
proper command-line flag, not via TF_XLA_FLAGS).


489/489 [==============================] - 4s 4ms/step - loss: 0.0096 - accuracy: 0.2740 - mape: 2363979.0000 - mse: 0.0039 - val_loss: 0.0084 - val_accuracy: 0.1585 - val_mape: 5152154.5000 - val_mse: 0.0040
Epoch 2/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0072 - accuracy: 0.2822 - mape: 5877780.0000 - mse: 0.0038 - val_loss: 0.0065 - val_accuracy: 0.3537 - val_mape: 6266235.0000 - val_mse: 0.0039
Epoch 3/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0056 - accuracy: 0.3640 - mape: 5755984.0000 - mse: 0.0038 - val_loss: 0.0051 - val_accuracy: 0.2073 - val_mape: 7118800.5000 - val_mse: 0.0039
Epoch 4/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0045 - accuracy: 0.2945 - mape: 6147206.0000 - mse: 0.0038 - val_loss: 0.0042 - val_accuracy: 0.3354 - val_mape: 7528637.5000 - val_mse: 0.0039
Epoch 5/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0038 - accuracy: 0.3804 - mape: 7003383.0000 - mse:

489/489 [==============================] - 2s 3ms/step - loss: 0.0023 - accuracy: 0.3476 - mape: 1979234.2500 - mse: 0.0029 - val_loss: 0.0030 - val_accuracy: 0.3293 - val_mape: 9383865.0000 - val_mse: 0.0036
Epoch 39/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0023 - accuracy: 0.3333 - mape: 2391753.2500 - mse: 0.0029 - val_loss: 0.0024 - val_accuracy: 0.4024 - val_mape: 1556693.8750 - val_mse: 0.0030
Epoch 40/400
489/489 [==============================] - 2s 4ms/step - loss: 0.0022 - accuracy: 0.3558 - mape: 1975076.5000 - mse: 0.0027 - val_loss: 0.0023 - val_accuracy: 0.3354 - val_mape: 902425.0625 - val_mse: 0.0030
Epoch 41/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0019 - accuracy: 0.3599 - mape: 1835998.8750 - mse: 0.0025 - val_loss: 0.0023 - val_accuracy: 0.3171 - val_mape: 2588944.2500 - val_mse: 0.0030
Epoch 42/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0020 - accuracy: 0.3722 - mape: 2284383.5000 - m

_________________________________________________________________
None
Epoch 1/400
489/489 [==============================] - 4s 4ms/step - loss: 0.0107 - accuracy: 0.2249 - mape: 4538028.0000 - mse: 0.0053 - val_loss: 0.0113 - val_accuracy: 0.1220 - val_mape: 9162100.0000 - val_mse: 0.0092
Epoch 2/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0085 - accuracy: 0.2965 - mape: 8831317.0000 - mse: 0.0052 - val_loss: 0.0095 - val_accuracy: 0.2866 - val_mape: 10376148.0000 - val_mse: 0.0093
Epoch 3/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0069 - accuracy: 0.3231 - mape: 9783632.0000 - mse: 0.0052 - val_loss: 0.0082 - val_accuracy: 0.5488 - val_mape: 9653076.0000 - val_mse: 0.0093
Epoch 4/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0057 - accuracy: 0.4356 - mape: 8897800.0000 - mse: 0.0052 - val_loss: 0.0072 - val_accuracy: 0.1951 - val_mape: 13703866.0000 - val_mse: 0.0092
Epoch 5/400
489/489 [======================

...

489/489 [==============================] - 2s 3ms/step - loss: 0.0017 - accuracy: 0.8487 - mape: 694479.5000 - mse: 0.0013 - val_loss: 0.0017 - val_accuracy: 0.8415 - val_mape: 94498.9062 - val_mse: 0.0015
Epoch 65/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0016 - accuracy: 0.8446 - mape: 373032.8750 - mse: 0.0010 - val_loss: 0.0018 - val_accuracy: 0.8354 - val_mape: 149227.2656 - val_mse: 0.0017
Epoch 66/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0020 - accuracy: 0.8691 - mape: 817628.4375 - mse: 0.0018 - val_loss: 0.0017 - val_accuracy: 0.8110 - val_mape: 326534.2500 - val_mse: 0.0015
Epoch 67/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0016 - accuracy: 0.8609 - mape: 358582.9688 - mse: 0.0012 - val_loss: 0.0014 - val_accuracy: 0.8476 - val_mape: 53123.4102 - val_mse: 7.2984e-04
Epoch 68/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0016 - accuracy: 0.8712 - mape: 466271.2500 - mse: 0.

Epoch 12/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0051 - accuracy: 0.2352 - mape: 2755237.7500 - mse: 0.0054 - val_loss: 0.0048 - val_accuracy: 0.2683 - val_mape: 3098952.5000 - val_mse: 0.0047
Epoch 13/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0050 - accuracy: 0.2393 - mape: 2599414.0000 - mse: 0.0052 - val_loss: 0.0053 - val_accuracy: 0.2012 - val_mape: 4327150.5000 - val_mse: 0.0054
Epoch 14/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0050 - accuracy: 0.2495 - mape: 2532071.7500 - mse: 0.0052 - val_loss: 0.0048 - val_accuracy: 0.2378 - val_mape: 4205824.0000 - val_mse: 0.0048
Epoch 15/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0047 - accuracy: 0.2065 - mape: 2519609.0000 - mse: 0.0049 - val_loss: 0.0051 - val_accuracy: 0.2805 - val_mape: 3060496.5000 - val_mse: 0.0051
Epoch 16/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0046 - accuracy: 0.2454 - mape: 26

Epoch 49/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0035 - accuracy: 0.2086 - mape: 2184707.0000 - mse: 0.0038 - val_loss: 0.0034 - val_accuracy: 0.2073 - val_mape: 3803288.2500 - val_mse: 0.0036
Epoch 50/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0036 - accuracy: 0.2352 - mape: 2222915.7500 - mse: 0.0040 - val_loss: 0.0035 - val_accuracy: 0.2317 - val_mape: 2015105.7500 - val_mse: 0.0038
Epoch 51/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0036 - accuracy: 0.2515 - mape: 2413703.0000 - mse: 0.0040 - val_loss: 0.0037 - val_accuracy: 0.2500 - val_mape: 3696201.7500 - val_mse: 0.0041
Epoch 52/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0036 - accuracy: 0.2168 - mape: 2136749.2500 - mse: 0.0041 - val_loss: 0.0033 - val_accuracy: 0.2256 - val_mape: 3509483.7500 - val_mse: 0.0035
Epoch 53/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0037 - accuracy: 0.2188 - mape: 24

489/489 [==============================] - 2s 3ms/step - loss: 0.0076 - accuracy: 0.2413 - mape: 9970104.0000 - mse: 0.0092 - val_loss: 0.0055 - val_accuracy: 0.1890 - val_mape: 8312569.0000 - val_mse: 0.0052
Epoch 11/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0078 - accuracy: 0.2188 - mape: 9764365.0000 - mse: 0.0094 - val_loss: 0.0052 - val_accuracy: 0.2317 - val_mape: 7544215.5000 - val_mse: 0.0049
Epoch 12/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0076 - accuracy: 0.2229 - mape: 10129861.0000 - mse: 0.0091 - val_loss: 0.0047 - val_accuracy: 0.1463 - val_mape: 11372980.0000 - val_mse: 0.0041
Epoch 13/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0074 - accuracy: 0.1963 - mape: 10300983.0000 - mse: 0.0089 - val_loss: 0.0062 - val_accuracy: 0.1524 - val_mape: 14010070.0000 - val_mse: 0.0064
Epoch 14/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0071 - accuracy: 0.1943 - mape: 10101950.00

Epoch 47/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0053 - accuracy: 0.2495 - mape: 7270815.5000 - mse: 0.0067 - val_loss: 0.0036 - val_accuracy: 0.2744 - val_mape: 7748715.0000 - val_mse: 0.0034
Epoch 48/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0055 - accuracy: 0.3027 - mape: 6096557.5000 - mse: 0.0069 - val_loss: 0.0038 - val_accuracy: 0.3354 - val_mape: 5834452.0000 - val_mse: 0.0038
Epoch 49/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0050 - accuracy: 0.3538 - mape: 4985927.0000 - mse: 0.0060 - val_loss: 0.0035 - val_accuracy: 0.3171 - val_mape: 7178753.0000 - val_mse: 0.0031
Epoch 50/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0052 - accuracy: 0.3374 - mape: 6042141.5000 - mse: 0.0064 - val_loss: 0.0037 - val_accuracy: 0.4146 - val_mape: 2434015.2500 - val_mse: 0.0035
Epoch 51/400
489/489 [==============================] - 2s 3ms/step - loss: 0.0055 - accuracy: 0.3129 - mape: 56

## Apply models

In [4]:
%%time 

for city, ini_date, end_date_train in zip(
    df.geocode, df.start_train_chik, df.end_train_chik): 
    

    # apply dl dengue on chik data 
    metrics = apply_model(city, ini_date = ini_date, 
                    end_date = end_date, look_back = LOOK_BACK, end_train_date =  '2022-11-01', batch_size = 1, 
                    predict_n = PREDICT_N,  ratio = None,
                    label_pred= 'chik_dengue_pred',
                    model_name = f'trained_{city}_dengue_msle', 
                    filename = f'../data/chik_{city}_cluster.csv.gz', plot = False)
    

    # apply transfer 

    metrics = apply_model(city, ini_date = ini_date, 
                    end_date = end_date, look_back = LOOK_BACK, end_train_date = '2022-11-01', batch_size = 1, 
                    predict_n = PREDICT_N,  ratio = None,
                    label_pred= 'chik_transf_pred',
                    model_name = f'trained_{city}_chik_transf_msle', 
                    filename = f'../data/chik_{city}_cluster.csv.gz', plot = False)
    

    # apply dl chik on chik 

    metrics = apply_model(city, ini_date = ini_date, 
                    end_date = end_date, look_back = LOOK_BACK, end_train_date =  '2022-11-01', batch_size = 1, 
                    predict_n = PREDICT_N,  ratio = None,
                    label_pred= 'chik_pred',
                    model_name = f'trained_{city}_chik_msle', 
                    filename = f'../data/chik_{city}_cluster.csv.gz', plot = False)
    


CPU times: user 1h 30min 5s, sys: 7min, total: 1h 37min 5s
Wall time: 1h 10min 34s
